<a href="https://colab.research.google.com/github/goga0001/graph/blob/main/minimal%20repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Manual Steps**

---




> Set the runtime to GPU via "Runtime > Change runtime type..."







# **Clone Git repository**

In [255]:
!git clone https://github.com/DeepGraphLearning/torchdrug

fatal: destination path 'torchdrug' already exists and is not an empty directory.


In [257]:
cd torchdrug

/content/torchdrug


# **Install requirements**

In [3]:
!pip install decorator  rdkit rdkit-pypi torch_geometric==2.0.4 matplotlib  tqdm  networkx   jinja2  lmdb  fair-esm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.1 MB 1.5 MB/s 
     |████████████████████████████████| 29.1 MB 83.8 MB/s 
     |████████████████████████████████| 407 kB 40.3 MB/s 
     |████████████████████████████████| 93 kB 838 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616602 sha256=9f27729c13a3b98d0e6b98f6b58598822ddc9b70b0b6b9d562add711bf61444a
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [4]:
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 8.9 MB 7.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 7.5 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.5 MB 7.2 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 722 kB 7.9 MB/s 


In [5]:
pip install fair-esm==0.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: fair-esm
    Found existing installation: fair-esm 2.0.0
    Uninstalling fair-esm-2.0.0:
      Successfully uninstalled fair-esm-2.0.0


In [6]:
!pip install Ninja==1.10.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 6.7 MB/s 


In [7]:
pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh

In [9]:
pip -V

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [10]:
 pip list | grep torch

torch                         1.12.1+cu113
torch-cluster                 1.6.0+pt112cu113
torch-geometric               2.0.4
torch-scatter                 2.1.0+pt112cu113
torch-sparse                  0.6.15+pt112cu113
torch-spline-conv             1.2.1+pt112cu113
torchaudio                    0.12.1+cu113
torchsummary                  1.5.1
torchtext                     0.13.1
torchvision                   0.13.1+cu113


# **Run setup.py**

In [ ]:
!python setup.py install

# **Prepare the Pretraining Dataset(5 min)**

 Before registering new dataset- upload csv(https://raw.githubusercontent.com/goga0001/graph/main/data.csv) into dataset folder

In [258]:
import os

from torchdrug import data, utils
from torchdrug.core import Registry as R
from collections import defaultdict

from torch.utils import data as torch_data

from torchdrug import data
from torchdrug.utils import doc


@R.register("datasets.WB6")
@utils.copy_args(data.MoleculeDataset.load_csv, ignore=("smiles_field", "target_fields"))
class WB6(data.MoleculeDataset):
    """
    Subset of random compound database for virtual screening.

    Statistics:
        - #Molecule:  4806
        - #Regression task: 2

    Parameters:
        path (str): path to store the dataset
        verbose (int, optional): output verbose level
        **kwargs
    """

    csv_file = "/content/torchdrug/torchdrug/datasets/data.csv"
    target_fields = ["logP","qed"]

    def __init__(self, path, verbose=1, **kwargs ):
        self.load_csv(self.csv_file, smiles_field="smiles", target_fields=self.target_fields,
                      verbose=verbose, **kwargs)

KeyError: ignored

In [ ]:
cd /content/torchdrug/torchdrug/datasets

/content/torchdrug/torchdrug/datasets


In [260]:
import torch
from torchdrug import core, models, tasks


dataset = WB("/content/torchdrug/torchdrug/datasets/data.csv", kekulize=True,
                            atom_feature="symbol")






Loading /content/torchdrug/torchdrug/datasets/data.csv: 4807it [00:00, 66041.32it/s]            
Constructing molecules from SMILES: 100%|██████████| 4806/4806 [00:09<00:00, 522.18it/s]


# **Define the Model: GraphAF (30sec)**

In [272]:
from torchdrug import core, models, tasks
from torchdrug.layers import distribution

model = models.RGCN(input_dim=dataset.num_atom_type,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256], batch_norm=True)

num_atom_type = dataset.num_atom_type
# add one class for non-edge
num_bond_type = dataset.num_bond_type + 1

node_prior = distribution.IndependentGaussian(torch.zeros(num_atom_type),
                                              torch.ones(num_atom_type))
edge_prior = distribution.IndependentGaussian(torch.zeros(num_bond_type),
                                              torch.ones(num_bond_type))
node_flow = models.GraphAF(model, node_prior, num_layer=12)
edge_flow = models.GraphAF(model, edge_prior, use_edge=True, num_layer=12)

task = tasks.AutoregressiveGeneration(node_flow, edge_flow,
                                      max_node=38, max_edge_unroll=12,
                                      criterion="nll")

In [273]:
dataset.num_bond_type

2

In [274]:
model.layers[0].num_relation

2

In [266]:
dataset[2]["graph"].num_relation

for data in dataset:
  data['graph'].num_relation = torch.tensor(2)

In [275]:
dataset[2]["graph"].num_relation

tensor(2)

# **Pretraining and Generation: GraphAF**

In [288]:
from torch import nn, optim
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=128, log_interval=10)

solver.train(num_epoch=1)
solver.save("graphaf_WB.pkl")

04:32:16   Preprocess training set


 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'eps': 1e-08,
               'foreach': None,
               'lr': 0.001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 5,
          'baseline_momentum': 0.9,
          'class': 'tasks.AutoregressiveGeneration',
          'criterion': 'nll',
          'edge_model': {'class': 'models.GraphAF',
                         'dequantization_noise': 0.9,
                         'model': {'activation': 'relu',
                                   'batch_norm': True,
                                   'class': 'models.RGCN',
                                   'concat_hidden': False,
                                   'edge_input_dim': None,

04:32:16   {'batch_size': 128,
 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'eps': 1e-08,
               'foreach': None,
               'lr': 0.001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 5,
          'baseline_momentum': 0.9,
          'class': 'tasks.AutoregressiveGeneration',
          'criterion': 'nll',
          'edge_model': {'class': 'models.GraphAF',
                         'dequantization_noise': 0.9,
                         'model': {'activation': 'relu',
                                   'batch_norm': True,
                                   'class': 'models.RGCN',
                                   'concat_hidden': False,
                           

04:32:16   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


04:32:16   Epoch 0 begin


RuntimeError: ignored

In [ ]:
from collections import defaultdict

solver.load("graphaf_WB.pkl")
results = task.generate(num_sample=32)
print(results.to_smiles())

17:17:52   Load checkpoint from graphaf_flavonoid_1epoch.pkl


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


['NCl', 'P', 'Br', 'ClP1(Cl)=P(P(Cl)(Cl)(Cl)P2=P3=S=2(Cl)S3(Cl)(Cl)(Cl)P(Cl)(Cl)(Cl)Cl)=P1', 'Cl', 'O', 'Cl[SH]12(OBr)P=P13PN32', 'P=[PH]1P23=P4(Cl)P5(Cl)=P6=S5(Cl)(Br)S612=P43Cl', 'ClP12(Cl)N=N[PH]1=N2', 'S', 'ClSBr', 'O', 'P', 'CBr', 'Cl', 'CCl', 'S', 'NBr', 'Cl', 'P', 'ClBr', 'PBr', 'ClCl', 'N', 'OCCl', 'N', 'P=S', 'Cl', 'S', 'SCl', 'O', 'C=O']
